# 发现群组

### 使用feedparser解析RSS订阅源

In [6]:
import feedparser
import re

# Returns title and dictionary of word counts for an RSS feed
def getwordcounts(url):
  # Parse the feed
  d=feedparser.parse(url)
  wc={}

  # Loop over all the entries
  for e in d.entries:
    if 'summary' in e: 
        summary=e.summary
    else: 
        summary=e.description

    # Extract a list of words
    words=getwords(e.title+' '+summary)
    for word in words:
      wc.setdefault(word,0)
      wc[word]+=1
  return d.feed.title,wc

def getwords(html):
  # Remove all the HTML tags
  txt=re.compile(r'<[^>]+>').sub('',html)

  # Split words by all non-alpha characters
  words=re.compile(r'[^A-Z^a-z]+').split(txt)

  # Convert to lowercase
  return [word.lower() for word in words if word!='']

### 解析并准备数据

In [10]:
apcount={}
wordcounts={}
feedlist=[line for line in open('feedlist.txt')]
for feedurl in feedlist:
  try:
    title,wc=getwordcounts(feedurl)
    wordcounts[title]=wc
    for word,count in wc.items():
      apcount.setdefault(word,0)
      if count>1:
        apcount[word]+=1
  except:
    print ('Failed to parse feed %s' % feedurl)

wordlist=[]
for w,bc in apcount.items():
  frac=float(bc)/len(feedlist)
  if frac>0.1 and frac<0.5:
    wordlist.append(w)

out=open('blogdata1.txt','w')
out.write('Blog')
for word in wordlist: out.write('\t%s' % word)
out.write('\n')

Failed to parse feed http://feeds.feedburner.com/37signals/beMH

Failed to parse feed http://feeds.feedburner.com/blogspot/bRuz

Failed to parse feed http://battellemedia.com/index.xml

Failed to parse feed http://feeds.searchenginewatch.com/sewblog

Failed to parse feed http://blog.topix.net/index.rdf

Failed to parse feed http://blogs.abcnews.com/theblotter/index.rdf

Failed to parse feed http://feeds.feedburner.com/ConsumingExperienceFull

Failed to parse feed http://flagrantdisregard.com/index.php/feed/

Failed to parse feed http://featured.gigaom.com/feed/

Failed to parse feed http://gofugyourself.typepad.com/go_fug_yourself/index.rdf

Failed to parse feed http://googleblog.blogspot.com/rss.xml

Failed to parse feed http://feeds.feedburner.com/GoogleOperatingSystem

Failed to parse feed http://headrush.typepad.com/creating_passionate_users/index.rdf

Failed to parse feed http://feeds.feedburner.com/instapundit/main

Failed to parse feed http://jeremy.zawodny.com/blog/rss2.xml

Fa

NameError: name 'file' is not defined

### 读取数据

In [1]:
def readfile(filename):
  lines=[line for line in open(filename)]
  
  # First line is the column titles
  colnames=lines[0].strip().split('\t')[1:]
  rownames=[]
  data=[]
  for line in lines[1:]:
    p=line.strip().split('\t')
    # First column in each row is the rowname
    rownames.append(p[0])
    # The data for this row is the remainder of the row
    data.append([float(x) for x in p[1:]])
  return rownames,colnames,data

### 皮尔逊相关度

In [2]:
from math import sqrt

def pearson(v1,v2):
  # Simple sums
  sum1=sum(v1)
  sum2=sum(v2)
  
  # Sums of the squares
  sum1Sq=sum([pow(v,2) for v in v1])
  sum2Sq=sum([pow(v,2) for v in v2])	
  
  # Sum of the products
  pSum=sum([v1[i]*v2[i] for i in range(len(v1))])
  
  # Calculate r (Pearson score)
  num=pSum-(sum1*sum2/len(v1))
  den=sqrt((sum1Sq-pow(sum1,2)/len(v1))*(sum2Sq-pow(sum2,2)/len(v1)))
  if den==0: return 0

  return 1.0-num/den

### 定义聚类class

In [3]:
class bicluster:
  def __init__(self,vec,left=None,right=None,distance=0.0,id=None):
    self.left=left
    self.right=right
    self.vec=vec
    self.id=id
    self.distance=distance

### 分级聚类

In [4]:
def hcluster(rows,distance=pearson):
  distances={}
  currentclustid=-1

  # Clusters are initially just the rows
  clust=[bicluster(rows[i],id=i) for i in range(len(rows))]

  while len(clust)>1:
    lowestpair=(0,1)
    closest=distance(clust[0].vec,clust[1].vec)

    # loop through every pair looking for the smallest distance
    for i in range(len(clust)):
      for j in range(i+1,len(clust)):
        # distances is the cache of distance calculations
        if (clust[i].id,clust[j].id) not in distances: 
          distances[(clust[i].id,clust[j].id)]=distance(clust[i].vec,clust[j].vec)

        d=distances[(clust[i].id,clust[j].id)]

        if d<closest:
          closest=d
          lowestpair=(i,j)

    # calculate the average of the two clusters
    mergevec=[
    (clust[lowestpair[0]].vec[i]+clust[lowestpair[1]].vec[i])/2.0 
    for i in range(len(clust[0].vec))]

    # create the new cluster
    newcluster=bicluster(mergevec,left=clust[lowestpair[0]],
                         right=clust[lowestpair[1]],
                         distance=closest,id=currentclustid)

    # cluster ids that weren't in the original set are negative
    currentclustid-=1
    del clust[lowestpair[1]]
    del clust[lowestpair[0]]
    clust.append(newcluster)

  return clust[0]

In [5]:
blognames, words, data = readfile('blogdata.txt')
clust = hcluster(data)
#printclust(clust, labels=blognames)

### 打印分级聚类层级结构

In [11]:
def printclust(clust,labels=None,n=0):
  # indent to make a hierarchy layout
  for i in range(n): print (' ', end='')
  if clust.id<0:
    # negative id means that this is branch
    print ('-')
  else:
    # positive id means that this is an endpoint
    if labels==None: print (clust.id)
    else: print (labels[clust.id])

  # now print the right and left branches
  if clust.left!=None: printclust(clust.left,labels=labels,n=n+1)
  if clust.right!=None: printclust(clust.right,labels=labels,n=n+1)

In [12]:
printclust(clust, labels=blognames)

-
 gapingvoid: "cartoons drawn on the back of business cards"
 -
  -
   Schneier on Security
   Instapundit.com
  -
   The Blotter
   -
    -
     MetaFilter
     -
      SpikedHumor
      -
       Captain's Quarters
       -
        Michelle Malkin
        -
         -
          NewsBusters.org - Exposing Liberal Media Bias
          -
           -
            Hot Air
            Crooks and Liars
           -
            Power Line
            Think Progress
         -
          Andrew Sullivan | The Daily Dish
          -
           Little Green Footballs
           -
            Eschaton
            -
             Talking Points Memo: by Joshua Micah Marshall
             Daily Kos
    -
     43 Folders
     -
      TechEBlog
      -
       -
        Mashable!
        Signum sine tinnitu--by Guy Kawasaki
       -
        -
         -
          Slashdot
          -
           MAKE Magazine
           Boing Boing
         -
          -
           Oilman
           -
            Online

### 绘制分级聚类的结果

In [13]:
from PIL import Image, ImageDraw

def getheight(clust):
  # Is this an endpoint? Then the height is just 1
  if clust.left==None and clust.right==None: return 1

  # Otherwise the height is the same of the heights of
  # each branch
  return getheight(clust.left)+getheight(clust.right)

def getdepth(clust):
  # The distance of an endpoint is 0.0
  if clust.left==None and clust.right==None: return 0

  # The distance of a branch is the greater of its two sides
  # plus its own distance
  return max(getdepth(clust.left),getdepth(clust.right))+clust.distance


def drawdendrogram(clust,labels,jpeg='clusters.jpg'):
  # height and width
  h=getheight(clust)*20
  w=1200
  depth=getdepth(clust)

  # width is fixed, so scale distances accordingly
  scaling=float(w-150)/depth

  # Create a new image with a white background
  img=Image.new('RGB',(w,h),(255,255,255))
  draw=ImageDraw.Draw(img)

  draw.line((0,h/2,10,h/2),fill=(255,0,0))    

  # Draw the first node
  drawnode(draw,clust,10,(h/2),scaling,labels)
  img.save(jpeg,'JPEG')

def drawnode(draw,clust,x,y,scaling,labels):
  if clust.id<0:
    h1=getheight(clust.left)*20
    h2=getheight(clust.right)*20
    top=y-(h1+h2)/2
    bottom=y+(h1+h2)/2
    # Line length
    ll=clust.distance*scaling
    # Vertical line from this cluster to children    
    draw.line((x,top+h1/2,x,bottom-h2/2),fill=(255,0,0))    
    
    # Horizontal line to left item
    draw.line((x,top+h1/2,x+ll,top+h1/2),fill=(255,0,0))    

    # Horizontal line to right item
    draw.line((x,bottom-h2/2,x+ll,bottom-h2/2),fill=(255,0,0))        

    # Call the function to draw the left and right nodes    
    drawnode(draw,clust.left,x+ll,top+h1/2,scaling,labels)
    drawnode(draw,clust.right,x+ll,bottom-h2/2,scaling,labels)
  else:   
    # If this is an endpoint, draw the item label
    draw.text((x+5,y-7),labels[clust.id],(0,0,0))

### 图片保存至本地

In [14]:
drawdendrogram(clust, blognames, jpeg='blogclust.jpg')

### 行和列对调，对单词聚类

In [15]:
def rotatematrix(data):
  newdata=[]
  for i in range(len(data[0])):
    newrow=[data[j][i] for j in range(len(data))]
    newdata.append(newrow)
  return newdata

In [16]:
rdata = rotatematrix(data)
wordclust = hcluster(rdata)
drawdendrogram(wordclust, labels=words, jpeg='wordclust.jpg')

### k均值聚类

In [18]:
import random

def kcluster(rows,distance=pearson,k=4):
  # Determine the minimum and maximum values for each point
  ranges=[(min([row[i] for row in rows]),max([row[i] for row in rows])) 
  for i in range(len(rows[0]))]

  # Create k randomly placed centroids
  clusters=[[random.random()*(ranges[i][1]-ranges[i][0])+ranges[i][0] 
  for i in range(len(rows[0]))] for j in range(k)]
  
  lastmatches=None
  for t in range(100):
    print('Iteration %d' % t)
    bestmatches=[[] for i in range(k)]
    
    # Find which centroid is the closest for each row
    for j in range(len(rows)):
      row=rows[j]
      bestmatch=0
      for i in range(k):
        d=distance(clusters[i],row)
        if d<distance(clusters[bestmatch],row): bestmatch=i
      bestmatches[bestmatch].append(j)

    # If the results are the same as last time, this is complete
    if bestmatches==lastmatches: break
    lastmatches=bestmatches
    
    # Move the centroids to the average of their members
    for i in range(k):
      avgs=[0.0]*len(rows[0])
      if len(bestmatches[i])>0:
        for rowid in bestmatches[i]:
          for m in range(len(rows[rowid])):
            avgs[m]+=rows[rowid][m]
        for j in range(len(avgs)):
          avgs[j]/=len(bestmatches[i])
        clusters[i]=avgs
      
  return bestmatches

In [20]:
kclust=kcluster(data, k=10)
print(kclust)
[blognames[r] for r in kclust[1]]

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
[[41, 42, 46, 81], [2, 7, 16, 23, 24, 25, 34, 36, 40, 48, 49, 56, 79, 80, 85, 91, 94], [27, 37, 51, 53, 64, 71, 90], [4, 5, 45], [11, 63, 82, 95], [6, 10, 14, 21, 26, 29, 59, 60, 69, 70, 73, 74, 75, 87, 92, 97], [0, 1, 3, 9, 15, 18, 19, 20, 22, 28, 31, 38, 39, 43, 47, 61, 72, 86, 88, 93, 98], [13, 32, 33, 35, 44, 54, 57, 62, 67, 76, 78, 83, 84, 96], [8, 12, 52, 65, 66, 68, 77], [17, 30, 50, 55, 58, 89]]


['Publishing 2.0',
 'GigaOM',
 "John Battelle's Searchblog",
 'Lifehacker',
 'Google Operating System',
 'Valleywag',
 'Topix.net Weblog',
 'Micro Persuasion',
 'Search Engine Watch Blog',
 'Techdirt',
 'Official Google Blog',
 'Search Engine Roundtable',
 'A Consuming Experience (full feed)',
 'Matt Cutts: Gadgets, Google, and SEO',
 'Quick Online Tips',
 'Google Blogoscoped',
 'Read/WriteWeb']

In [23]:
# from BeautifulSoup import BeautifulSoup
# import urllib2
# import re
# chare=re.compile(r'[!-\.&]')
# itemowners={}

# # Words to remove
# dropwords=['a','new','some','more','my','own','the','many','other','another']

# currentuser=0
# for i in range(1,51):
#   # URL for the want search page
#   c=urllib2.urlopen(
#   'http://member.zebo.com/Main?event_key=USERSEARCH&wiowiw=wiw&keyword=car&page=%d'
#   % (i))
#   soup=BeautifulSoup(c.read())
#   for td in soup('td'):
#     # Find table cells of bgverdanasmall class
#     if ('class' in dict(td.attrs) and td['class']=='bgverdanasmall'):
#       items=[re.sub(chare,'',str(a.contents[0]).lower()).strip() for a in td('a')]
#       for item in items:
#         # Remove extra words
#         txt=' '.join([t for t in item.split(' ') if t not in dropwords])
#         if len(txt)<2: continue
#         itemowners.setdefault(txt,{})
#         itemowners[txt][currentuser]=1
#       currentuser+=1
      
# out=open('zebo.txt','w')
# out.write('Item')
# for user in range(0,currentuser): out.write('\tU%d' % user)
# out.write('\n')
# for item,owners in itemowners.items():
#   if len(owners)>10:
#     out.write(item)
#     for user in range(0,currentuser):
#       if user in owners: out.write('\t1')
#       else: out.write('\t0')
#     out.write('\n')

In [26]:
def tanamoto(v1,v2):
  c1,c2,shr=0,0,0
  
  for i in range(len(v1)):
    if v1[i]!=0: c1+=1 # in v1
    if v2[i]!=0: c2+=1 # in v2
    if v1[i]!=0 and v2[i]!=0: shr+=1 # in both
  
  return 1.0-(float(shr)/(c1+c2-shr))

In [28]:
wants, people, data = readfile('zebo.txt')
clust = hcluster(data, distance=tanamoto)
drawdendrogram(clust, wants)

In [31]:
def scaledown(data,distance=pearson,rate=0.01):
  n=len(data)

  # The real distances between every pair of items
  realdist=[[distance(data[i],data[j]) for j in range(n)] 
             for i in range(0,n)]

  # Randomly initialize the starting points of the locations in 2D
  loc=[[random.random(),random.random()] for i in range(n)]
  fakedist=[[0.0 for j in range(n)] for i in range(n)]
  
  lasterror=None
  for m in range(0,1000):
    # Find projected distances
    for i in range(n):
      for j in range(n):
        fakedist[i][j]=sqrt(sum([pow(loc[i][x]-loc[j][x],2) 
                                 for x in range(len(loc[i]))]))
  
    # Move points
    grad=[[0.0,0.0] for i in range(n)]
    
    totalerror=0
    for k in range(n):
      for j in range(n):
        if j==k: continue
        # The error is percent difference between the distances
        errorterm=(fakedist[j][k]-realdist[j][k])/realdist[j][k]
        
        # Each point needs to be moved away from or towards the other
        # point in proportion to how much error it has
        grad[k][0]+=((loc[k][0]-loc[j][0])/fakedist[j][k])*errorterm
        grad[k][1]+=((loc[k][1]-loc[j][1])/fakedist[j][k])*errorterm

        # Keep track of the total error
        totalerror+=abs(errorterm)
    print (totalerror)

    # If the answer got worse by moving the points, we are done
    if lasterror and lasterror<totalerror: break
    lasterror=totalerror
    
    # Move each of the points by the learning rate times the gradient
    for k in range(n):
      loc[k][0]-=rate*grad[k][0]
      loc[k][1]-=rate*grad[k][1]

  return loc

def draw2d(data,labels,jpeg='mds2d.jpg'):
  img=Image.new('RGB',(2000,2000),(255,255,255))
  draw=ImageDraw.Draw(img)
  for i in range(len(data)):
    x=(data[i][0]+0.5)*1000
    y=(data[i][1]+0.5)*1000
    draw.text((x,y),labels[i],(0,0,0))
  img.save(jpeg,'JPEG')  

In [36]:
blognames, words, data = readfile('blogdata.txt')
coords = scaledown(data, rate=0.01)
draw2d(coords, blognames, jpeg='blogs2d.jpg')

4296.813892193015
3628.4864275905925
3539.9081903644365
3496.6166105924863
3462.7290404176338
3434.0328664143385
3408.236895515862
3383.3359804875063
3360.7886350498316
3336.1964089267813
3313.4711976526255
3295.224475023654
3281.0366360090225
3269.6265717178503
3259.4811554606395
3250.389440066609
3241.0664667185256
3231.575993216872
3222.268863578611
3213.701241084538
3205.189172449979
3197.3486443875404
3189.3872718758
3180.9782691460023
3173.161101086121
3166.2969224747035
3159.9086130969404
3153.920010543945
3148.0167634208096
3142.4107270953377
3137.0723402399854
3132.1321180264813
3127.244744666289
3122.166704704863
3116.7227530286677
3112.240840396887
3108.6612665322486
3105.091928844856
3101.5620024067334
3098.5136870516308
3095.212774805188
3092.012882963618
3089.2729808240756
3086.8356468842285
3084.5255797492327
3082.414360715347
3080.3366120191386
3078.377211447293
3076.6835619171598
3075.105322679719
3073.5661928330665
3072.118009733593
3070.6632880113502
3069.01033094726